# Tesserae v5 Demo

This demo will go over the basics of Tesserae v5 development up through October 11, 2018.

In [1]:
import json

from tesserae.db import TessMongoConnection
from tesserae.db.entities import Frequency, Match, Text, Token, Unit
from tesserae.utils import TessFile
from tesserae.tokenizers import GreekTokenizer, LatinTokenizer
from tesserae.unitizer import Unitizer
from tesserae.matchers import AggregationMatcher

# Set up the connection and clean up the database
connection = TessMongoConnection('45.55.219.221', 27017, None, None, 'tesstest')

# Clean up the previous demo
connection.connection['feature_sets'].delete_many({})
connection.connection['frequencies'].delete_many({})
connection.connection['matches'].delete_many({})
connection.connection['match_sets'].delete_many({})
connection.connection['texts'].delete_many({})
connection.connection['tokens'].delete_many({})
connection.connection['units'].delete_many({})

## Loading and Storing New Texts

The Tesserae database catalogs metadata, including the title, author, and year published, as well as integrity information like filepath, MD5 hash, and CTS URN.

We start by loading in some metadata from `text_metadata.json`.

In [2]:
with open('text_metadata.json', 'r') as f:
    text_meta = json.load(f)

print('{}{}{}{}'.format('Title'.ljust(15), 'Author'.ljust(15), 'Language'.ljust(15), 'Year'))
print('{}{}{}{}'.format('-----'.ljust(15), '------'.ljust(15), '--------'.ljust(15), '----'))
for t in text_meta:
    print('{}{}{}{}'.format(t['title'].ljust(15), t['author'].ljust(15), t['language'].ljust(15), str(t['year']).ljust(15)))

Title          Author         Language       Year
-----          ------         --------       ----
aeneid         vergil         latin          19             
de oratore     cicero         latin          38             
heracles       euripides      greek          -416           
epistles       plato          greek          -280           


Then insert the new texts with `TessMongoConnection.insert` after converting the raw JSON to Tesserae `Text` entities.

In [3]:
texts = []
for t in text_meta:
    texts.append(Text.json_decode(t))
result = connection.insert(texts)
print('Inserted {} texts.'.format(len(result.inserted_ids)))
print(result.inserted_ids)

Inserted 4 texts.
[ObjectId('5c4a0011087a57233b14a6ea'), ObjectId('5c4a0011087a57233b14a6eb'), ObjectId('5c4a0011087a57233b14a6ec'), ObjectId('5c4a0011087a57233b14a6ed')]


We can retrieve the inserted texts with `TessMongoConnection.find`. These texts will be converted to objects representing the database entries. The returned text list can be filtered by any valid field in the text database.

In [4]:
texts = connection.find('texts', _id=result.inserted_ids)

print('{}{}{}{}'.format('Title'.ljust(15), 'Author'.ljust(15), 'Language'.ljust(15), 'Year'))
for t in texts:
    print('{}{}{}{}'.format(t.title.ljust(15), t.author.ljust(15), t.language.ljust(15), t.year))

Title          Author         Language       Year
aeneid         vergil         latin          19
de oratore     cicero         latin          38
heracles       euripides      greek          -416
epistles       plato          greek          -280


## Loading .tess Files

Text metadata includes the path to the .tess file on the local filesystem. Using a Text retrieved from the database, the file can be loaded for further processing.

In [5]:
tessfile = TessFile(texts[0].path, metadata=texts[0])

print(tessfile.path)
print(len(tessfile))
print(tessfile[270])

la/vergil.aeneid.tess
9908
<verg. aen. 1.271>	transferet, et longam multa vi muniet Albam.



We can iterate through the file line-by-line.

In [6]:
lines = tessfile.readlines()
for i in range(10):
    print(next(lines))

<verg. aen. 1.1>	Arma virumque cano, Troiae qui primus ab oris

<verg. aen. 1.2>	Italiam, fato profugus, Laviniaque venit

<verg. aen. 1.3>	litora, multum ille et terris iactatus et alto

<verg. aen. 1.4>	vi superum saevae memorem Iunonis ob iram;

<verg. aen. 1.5>	multa quoque et bello passus, dum conderet urbem,

<verg. aen. 1.6>	inferretque deos Latio, genus unde Latinum,

<verg. aen. 1.7>	Albanique patres, atque altae moenia Romae.

<verg. aen. 1.8>	Musa, mihi causas memora, quo numine laeso,

<verg. aen. 1.9>	quidve dolens, regina deum tot volvere casus

<verg. aen. 1.10>	insignem pietate virum, tot adire labores



We can also iterate token-by-token.

In [7]:
tokens = tessfile.read_tokens()
for i in range(10):
    print(next(tokens))

Arma
virumque
cano,
Troiae
qui
primus
ab
oris
Italiam,
fato


## Tokenizing a Text

Texts can be tokenized with `tesserae.tokenizers.get_token_info`. This function takes a token and the language to use for lemmatization, etc.

In [8]:
tokenizer = GreekTokenizer(connection) if tessfile.metadata.language == 'greek' else LatinTokenizer(connection)

tokens, frequencies, feature_sets = tokenizer.tokenize(tessfile.read(), text=tessfile.metadata)

tokens = tokenizer.tokens
print(len(tokens), len(frequencies), len(feature_sets))

print('{}{}{}{}'.format('Raw'.ljust(15), 'Normalized'.ljust(15), 'Lemmata'.ljust(15), 'Frequency'))
print('{}{}{}{}'.format('---'.ljust(15), '----------'.ljust(15), '-------'.ljust(15), '---------'))
for i in range(20):
    if tokenizer.tokens[i].feature_set is not None:
        print('{}{}{}{}'.format(tokenizer.tokens[i].display.ljust(15),
                              str(tokenizer.tokens[i].feature_set.form).ljust(15),
                              str(tokenizer.tokens[i].feature_set.lemmata).ljust(15),
                              tokenizer.tokens[i].frequency.frequency))


140667 16602 16602
Raw            Normalized     Lemmata        Frequency
---            ----------     -------        ---------
Arma           arma           ['arma', 'armo']148
virumque       uirumque       ['uir', 'uirus']4
cano           cano           ['canus', 'cano']3
Troiae         troiae         ['troius', 'troia']36
qui            qui            ['quis', 'qui']166
primus         primus         ['primus']     34
ab             ab             ['ab']         136
oris           oris           ['ora', 'os', 'aurum']32
Italiam        italiam        ['italia']     32


Processed tokens can then be stored in and retrieved from the database, similar to text metadata.

In [9]:
result = connection.insert(feature_sets)
print('Inserted {} feature set entities out of {}'.format(len(result.inserted_ids), len(feature_sets)))

result = connection.insert(frequencies)
print('Inserted {} frequency entities out of {}'.format(len(result.inserted_ids), len(frequencies)))

# result = connection.insert(tokens)
# print('Inserted {} tokens out of {}.'.format(len(result.inserted_ids), len(tokens)))

Inserted 16602 feature set entities out of 16602
Inserted 16602 frequency entities out of 16602


## Unitizing a Text

Texts can be unitized into lines and phrases, and the intertext matches are found between units of text.


In [10]:
# Unitizing lines of a poem
unitizer = Unitizer()
lines, phrases = unitizer.unitize(tokens, tessfile.metadata)

print('Lines\n-----')
for line in lines[:20]:
        print(''.join([t.display for t in line.tokens]))
        
print('\n\nPhrases\n-------')
for phrase in phrases[:20]:
        print(''.join([t.display for t in phrase.tokens]))

Lines
-----
Arma virumque cano, Troiae qui primus ab oris / 
Italiam, fato profugus, Laviniaque venit / 
litora, multum ille et terris iactatus et alto / 
vi superum saevae memorem Iunonis ob iram; / 
multa quoque et bello passus, dum conderet urbem, / 
inferretque deos Latio, genus unde Latinum, / 
Albanique patres, atque altae moenia Romae. / 
Musa, mihi causas memora, quo numine laeso, / 
quidve dolens, regina deum tot volvere casus / 
insignem pietate virum, tot adire labores / 
impulerit. Tantaene animis caelestibus irae? / 
Urbs antiqua fuit, Tyrii tenuere coloni, / 
Karthago, Italiam contra Tiberinaque longe / 
ostia, dives opum studiisque asperrima belli; / 
quam Iuno fertur terris magis omnibus unam / 
posthabita coluisse Samo; hic illius arma, / 
hic currus fuit; hoc regnum dea gentibus esse, / 
si qua fata sinant, iam tum tenditque fovetque. / 
Progeniem sed enim Troiano a sanguine duci / 
audierat, Tyrias olim quae verteret arces; / 


Phrases
-------
Arma virumque cano, Tr

In [11]:
# Unitizing phrases of a poem or prose
result = connection.insert(lines + phrases)
print('Inserted {} units out of {}.'.format(len(result.inserted_ids), len(lines + phrases)))

result = connection.insert(tokens)
print('Inserted {} tokens out of {}.'.format(len(result.inserted_ids), len(tokens)))

Inserted 14997 units out of 14997.
Inserted 140667 tokens out of 140667.


In [12]:
for text in texts[1:]:
    tessfile = TessFile(text.path, metadata=text)
    tokenizer = GreekTokenizer(connection) if tessfile.metadata.language == 'greek' else LatinTokenizer(connection)

    
    tokens, frequencies, feature_sets = tokenizer.tokenize(tessfile.read(), text=tessfile.metadata)
        
    tokens = tokenizer.tokens
    result = connection.insert(feature_sets)
    result = connection.insert(frequencies)
    
    unitizer = Unitizer()
    lines, phrases = unitizer.unitize(tokens, tessfile.metadata)
    result = connection.insert(lines + phrases)
    
    result = connection.insert(tokens)

## Matching

Once the Texts, Tokens, and Units are in the database, we can then find intertext matches.

In [13]:
import time
matcher = AggregationMatcher(connection)
match_texts = [t for t in texts if t.language == 'latin']

start = time.time()
matches, match_set = matcher.match(match_texts, 'phrase', 'form', distance_metric='frequency', max_distance=999)
print("Completed matching in {0:.2f}s".format(time.time() - start))

matches.sort(key=lambda x: x.score, reverse=True)

result = connection.insert(match_set)
print('Inserted {} match set entities out of {}'.format(len(result.inserted_ids), 1))
result = connection.insert(matches)
print('Inserted {} match entities out of {}'.format(len(result.inserted_ids), len(matches)))

Using corpus
[ObjectId('5c4a0022087a57233b17ae96'), ObjectId('5c4a0022087a57233b17ae9d'), ObjectId('5c4a0022087a57233b17ae9c'), ObjectId('5c4a0022087a57233b17ae9b'), ObjectId('5c4a0022087a57233b17ae9a'), ObjectId('5c4a0022087a57233b17ae99'), ObjectId('5c4a0022087a57233b17ae98'), ObjectId('5c4a0022087a57233b17ae97'), ObjectId('5c4a0022087a57233b17ae35'), ObjectId('5c4a0022087a57233b17ae34')]


KeyboardInterrupt: 

In [ ]:
matches = connection.aggregate('matches', [
    {'$match': {'match_set': match_set.id}},
    {'$lookup': {
        'from': 'units',
        'let': {'m_units': '$units'},
        'pipeline': [
            {'$match': {'$in':['$_id', '$$m_units']}},
            {'$lookup': {
                'from': 'tokens',
                'let': {'token_ids': '$tokens'},
                'pipeline': [
                    {'$match': {'$in': ['$_id', '$$token_ids']}},
                    {'$sort': {'$index': 1}}
                ],
                'as': 'tokens'
            }},
        ],
        'as': 'units'
    }},
    {'$sort': {'score': -1}}
])


print('\n')
print('{}{}{}'.format('Score'.ljust(15), 'Match Tokens'.ljust(15), 'Source Text'.ljust(15), 'Target Text'))
print('{}{}{}'.format('-----'.ljust(15), '------------'.ljust(15), '-----------'.ljust(15), '-----------'))
for m in matches:
    print('{}{}'.format(('%.3f'%(m.score)).ljust(15), ', '.join(list(set([t.display for t in m.tokens[0]])))))
    print('Source Text: {}'.format(''.join([tokens[0][t].display for t in m.units[0].tokens])))
    print('Target Text: {}'.format(''.join([tokens[1][t].display for t in m.units[1].tokens])))
    print('\n')